In [14]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [15]:
# Load data
df = pd.read_csv("/Users/parakhchaudhary/SHL_Recommendation_System/dataset/dataset_final.csv")
embeddings = np.load("/Users/parakhchaudhary/SHL_Recommendation_System/dataset/dataset_final.npy")

# Load model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Parameters
SIMILARITY_THRESHOLD = 0.4
MAX_RESULTS = 10
MIN_RESULTS = 1


/Users/parakhchaudhary/opt/anaconda3/envs/chaenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:

def recommend_tests(user_query):
    query_embedding = model.encode([user_query])
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    
    # Filter by threshold
    matching_indices = np.where(similarities >= SIMILARITY_THRESHOLD)[0]
    
    if len(matching_indices) == 0:
        # fallback to top-1 if nothing matches
        matching_indices = np.array([np.argmax(similarities)])
    
    # Sort by similarity
    sorted_indices = matching_indices[np.argsort(similarities[matching_indices])[::-1]]
    top_indices = sorted_indices[:MAX_RESULTS]
    
    # Return results
    results = df.iloc[top_indices].copy()
    results["similarity"] = similarities[top_indices]

    print("Top similarity scores:", sorted(similarities, reverse=True)[:5])
    print(f"Matched indices (≥ {SIMILARITY_THRESHOLD}):", matching_indices)

    return results[["name", "test_type", "url", "similarity"]]

In [17]:
# 🔍 Example Query
query = "ICICI Bank Assistant Admin, Experience required 0-2 years, test should be 30-40 mins long"
recommendations = recommend_tests(query)

print("\n✅ Recommended Tests:")
for name in recommendations["name"].to_list():
    print("-", name)


Top similarity scores: [0.5914551, 0.547591, 0.53832364, 0.48696387, 0.4781316]
Matched indices (≥ 0.4): [  7   8   9  10  11  14  18  46  50  55  56  58  66  70  75  76  81  88
  89  90  91  92  94  99 100 101 103 104 105 110 111 112 113 115 126 131
 135 138 141 143 144 145 146 147 160 168 172 174 191 192 195 205 215 240
 241 242 247 250 251 252 253 254 263 265 266 267 278 279 283 290 291 292
 308 313 318 319 320 323 335 336 337 338 340 347 351 352 353 354 367 372
 373 374]

✅ Recommended Tests:
- Financial and Banking Services (New)
- Financial Accounting (New)
- Accounts Payable (New)
- Pega Development (New)
- Accounts Receivable (New)
- Entry Level Cashier Solution
- Software Business Analysis
- Accounts Payable Simulation (New)
- Economics (New)
- Global Skills Assessment
